In [ ]:
import requests  # For HTTP requests
import re  # For regular expressions
import os  # For interacting with the operating system
import time  # For time-related operations
import json  # For handling JSON data
import shutil  # For file operations like copy, move, delete, etc.
import pdfplumber  # For extracting content from PDF files
import pandas as pd  # For data manipulation and analysis
from openpyxl import Workbook  # For creating and manipulating Excel workbooks
from urllib.parse import urlparse  # For parsing URLs


# Mapping of document types to their corresponding names in the World Bank API
FileNameConvertDict = {
    'icrr': 'Implementation+Completion+Report+Review',
    'icr': "Implementation+Completion+and+Results+Report",
    'ppar': "Project+Performance+Assessment+Report",
    'pad': 'Project+Appraisal+Document',
    'scd': 'Systematic+Country+Diagnostic',
    'isr': 'Implementation+Status+and+Results+Report',
    'pd': 'Program+Document',
    'esmp': 'Environmental+and+Social+Management+Plan',
    'cas': 'Country+Assistance+Strategy+Document',
    'cpf': 'Country+Partnership+Framework',
    'cpe': 'IEG+Evaluation',
    'cen': 'Country+Engagement+Note',
    'cren': 'Country+Re-engagement+Note',
    'isn': 'Interim+Strategy+Note',
    'cpe': 'Country+Program+Evaluation',
    'asa': 'Economic+%26+Sector+Work%5EEconomic+%26amp%3B+Sector+Work%5EPublications%5EPublications+%26+Research%5EPublications+%26amp%3B+Research',
    'clrr': 'CAS+Completion+Report+Review',
    'pp': 'Project+Paper',
}



        
class URLBulkDownload(object):
    """
    A class to download multiple files from URLs.

    Args:
        input_dict (dict): A dictionary containing the necessary input parameters.
                           Required key: 'root_path' (string) - The root path where downloaded files will be stored.
                           Optional key: 'urls' (list) - A list of URLs to download files from (default is an empty list).
                           Optional key: 'download_version' (string) - The file download version ('txt' or 'binary')
                                                                       (default is 'txt').

    Attributes:
        root_path (string): The root path where downloaded files will be stored.
        urls (list): A list of URLs to download files from.
        download_version (string): The file download version ('txt' or 'binary').
        output_path (string): The path where downloaded files will be saved under the 'URL_downloads' directory.
    """
    def __init__(self, input_dict):
        self.root_path = input_dict['root_path']
        self.urls = input_dict.get('urls', [])
        self.download_version = input_dict.get('download_version', 'txt')
        self.output_path = os.path.join(self.root_path, "URL_downloads")
        os.makedirs(self.output_path, exist_ok=True)

    def download_files(self):
        """
        Downloads files from the URLs specified in the 'urls' attribute.

        It saves the downloaded files in either text or binary format based on the 'download_version' attribute.

        Returns:
            Tuple: A tuple containing two lists - downloaded URLs and failed URLs.
        """
        downloaded_urls = []
        failed_urls = []
        invalid_content_type_urls = []

        print(f"Start downloading {len(self.urls)} files using URLs...")
        for url in self.urls:
            file_name = os.path.basename(url)
            try:
                response = requests.get(url)
                if response.status_code == 200:
                    # Get the content type from the response headers
                    content_type = response.headers.get('content-type', '').lower()

                    if 'pdf' in content_type or 'text/plain' in content_type:
                        if self.download_version == 'txt':
                            file_path = os.path.join(self.output_path, file_name)
                            with open(file_path, 'w', encoding='utf-8', errors='ignore') as f:
                                f.write(response.text)
                        else:
                            file_path = os.path.join(self.output_path, file_name)
                            with open(file_path, 'wb') as f:
                                f.write(response.content)
                        print(f"File downloaded: {file_name}")
                        downloaded_urls.append(url)
                    else:
                        print(f"Invalid content type for {file_name}. Content type: {content_type}")
                        invalid_content_type_urls.append(url)
                else:
                    print(f"Failed to download {file_name}. Status code: {response.status_code}")
                    failed_urls.append(url)
            except Exception as e:
                print(f"Error downloading {file_name}: {e}")
                failed_urls.append(url)

        return downloaded_urls, failed_urls, invalid_content_type_urls

    def run(self):
        """
        Run the download_files method to start the bulk download process.
        """
        self.download_files()

    def __call__(self):
        """
        Allows the object to be called like a function, executing the 'run' method.
        """
        return self.run()
        
# Read URLs from Excel
def read_urls_from_excel(excel_path, column_name='URL'):
    try:
        df = pd.read_excel(excel_path)
        return df[column_name].dropna().tolist()
    except Exception as e:
        print(f"Error reading Excel file: {e}")
        return []

global_urls_dict = {}



# Function to initiate the file download and create the Excel summary
if __name__ == '__main__':
    global global_folder_path, global_urls_dict  # Use the global variables
    global_folder_path = ''# Path to save the downloaded files
    excel_path = '.xlsx'  # Excel containing all the URLs to be downloaded (Column name shall be URL)
    folder_path = global_folder_path  # Use the global variable
    urls = []
    
    urls = read_urls_from_excel(excel_path, column_name='URL')

 

    # Store URLs in the global dictionary
    global_urls_dict[folder_path] = urls

    # Check if the data folder exists, if not create it
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    input_dict = {
        'root_path': folder_path,
        'urls': urls,
        'download_version': 'binary'  # Download files in binary format (pdf or txt)
    }
    bulk_downloader = URLBulkDownload(input_dict)
    bulk_downloader.download_files()
    


# The following part is for renaming the files in the standard Format and save the Project Ids of the downloaded files in a excel file

# Define the mapping of phrases to their abbreviations
filetype_mapping = {
    'Implementation Completion Report Review': 'ICRR',
    'Implementation Completion and Results Report': 'ICR',
    'Project Performance Assessment Report': 'PPAR',
    'Project Appraisal Document': 'PAD',
    'Systematic Country Diagnostic': 'SCD',
    'Implementation Status and Results Report': 'ISR',
    'Program Document': 'PD',
    'Environmental and Social Management Plan': 'ESMP',
    'Country Assistance Strategy Document': 'CAS',
    'Country Partnership Framework': 'CPF',
    'IEG Evaluation': 'CPE',
    'Country Engagement Note': 'CEN',
    'Country Re-engagement Note': 'CREN',
    'Interim Strategy Note': 'ISN',
    'Country Program Evaluation': 'CPE',
    'ASA': 'ASA',
    'CAS Completion Report Review': 'CLRR',
    'Project Paper': 'PP',
    'Restructing Paper': 'RES'
}

# Iterate through folder paths and their associated URLs
for folder_path, urls in global_urls_dict.items():
    for url in urls:
        # Extract filename from URL
        filename = os.path.basename(urlparse(url).path)

        # Construct the full file path
        file_path = os.path.join(folder_path, "URL_downloads", filename)

        # Initialize content variable
        content = ""

        # Read the content of the file
        file_extension = os.path.splitext(file_path)[1].lower()
    
        # Handle PDF files using PyPDF2
        if file_extension == '.pdf':
            with pdfplumber.open(file_path) as pdf:
                content = ''
                start = 0  # Specify the start page index
                end = 5    # Specify the end page index
                pdf_pages = pdf.pages[start:end]

                for pdf_page in pdf_pages:
                    page_text = ''

                    # Extract header content (top portion of the page)
                    header_height = 100  # Adjust this value based on your PDF's header height
                    header_area = pdf_page.crop((0, 0, pdf_page.width, header_height))
                    header_text = header_area.extract_text()

                    # Extract footer content (bottom portion of the page)
                    footer_height = 100  # Adjust this value based on your PDF's footer height
                    footer_area = pdf_page.crop((0, pdf_page.height - footer_height, pdf_page.width, pdf_page.height))
                    footer_text = footer_area.extract_text()

                    # Extract main content (excluding header and footer)
                    main_area = pdf_page.crop((0, header_height, pdf_page.width, pdf_page.height - footer_height))
                    main_text = main_area.extract_text()

                    # Combine header, main, and footer content
                    page_text = f"{header_text}\n\n{main_text}\n\n{footer_text}\n\n\x0c\n\n"

                    content += page_text
            
        else:
            with open(file_path, "r") as text_file:
                content = text_file.read()


        # Extract "projectid" from the content
        projectid_matches = re.findall(r'P\d{6}', content)
        extracted_projectid = projectid_matches[0] if projectid_matches else ""

        # Extract "filetype" from the filename or content
        filetype_match = re.search(r'-(ICRR|ICR|PPAR|PAD|SCD|ISR|PD|ESMP|CAS|CPF|CPE|CEN|CREN|ISN|CPE|ASA|CLRR|PP)-', url)

        # If no direct match from filename, check content
        if not filetype_match:
            patterns = [r'Report No:\s*(ICRR|RES|ICR|PPAR|PAD|SCD|ISR|PD|ESMP|CAS|CPF|CPE|CEN|CREN|ISN|CPE|ASA|CLRR|PP)',
                       r'Report Number:\s*(ICRR|RES|ICR|PPAR|PAD|SCD|ISR|PD|ESMP|CAS|CPF|CPE|CEN|CREN|ISN|CPE|ASA|CLRR|PP)',
                       r'Report Number\s*:\s*(ICRR|RES|ICR|PPAR|PAD|SCD|ISR|PD|ESMP|CAS|CPF|CPE|CEN|CREN|ISN|CPE|ASA|CLRR|PP)',
                       r'Report No.:\s*(ICRR|RES|ICR|PPAR|PAD|SCD|ISR|PD|ESMP|CAS|CPF|CPE|CEN|CREN|ISN|CPE|ASA|CLRR|PP)'
                      ]
            for pattern in patterns:
                filetype_match = re.search(pattern, content)
                if filetype_match:
                    break  # Stop searching if a match is found

        # If still no match, check for specific phrases in the content
        if not filetype_match:
            phrase_frequencies = {phrase: content.lower().count(phrase.lower()) for phrase in filetype_mapping.keys()}
            most_common_phrase = max(phrase_frequencies, key=phrase_frequencies.get)

            if phrase_frequencies[most_common_phrase] > 0:
                extracted_filetype = filetype_mapping[most_common_phrase]
            else:
                extracted_filetype = "UNKNOWN"
        else:
            extracted_filetype = filetype_match.group(1)  # Extracted directly from the match

        # Extract "guid" from the URL
        guid_match = re.search(r'/(\d+)/', urlparse(url).path)
        extracted_guid = guid_match.group(1) if guid_match else ""


        # Rename the file if necessary
        if extracted_projectid and extracted_guid and extracted_filetype:
            # Get the original file extension
            _, original_extension = os.path.splitext(filename)

            # Construct the new filename
            new_filename = f"{extracted_projectid}_{extracted_filetype}_{extracted_guid}{original_extension}"

            # Construct the paths for renaming
            renamed_path = os.path.join(folder_path, "URL_downloads", new_filename)

            # Rename the file
            os.rename(file_path, renamed_path)


# Provide the folder path where downloaded files are located
folder_path2 = f"{folder_path}/URL_downloads"

# Provide the folder path where you want to save the Excel file
output_folder = f"{folder_path}"

## Create a new Excel workbook
workbook = Workbook()
sheet = workbook.active
sheet.title = "Downloaded Files"  # Set the sheet name

# Set the column headers
sheet["A1"] = "Proj.Id"
sheet["B1"] = "File Type"

# Initialize row counter
row_num = 2

# List all files in the folder
file_list = os.listdir(folder_path2)

# Iterate through the files and extract project ID and file type
for filename in file_list:
    if filename != ".DS_Store" and "_" in filename and "." in filename:
        parts = filename.split("_")
        project_id = parts[0]
        file_type = parts[1]
        sheet.cell(row=row_num, column=1, value=project_id)
        sheet.cell(row=row_num, column=2, value=file_type)
        row_num += 1

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Save the Excel workbook in the output folder
excel_filename = os.path.join(output_folder, f"Downloaded_Project_IDs.xlsx")
workbook.save(excel_filename)

print(f"Download Summary saved as '{excel_filename}'")
